In [13]:
# %%
import re
import os
from langchain_community.document_loaders import TextLoader
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain_qdrant import QdrantVectorStore
from langchain.text_splitter import (
    MarkdownHeaderTextSplitter,
    RecursiveCharacterTextSplitter
)
from langchain.schema import Document
from typing import List, Dict

from dotenv import load_dotenv

load_dotenv()

True

In [14]:
class Config:
    GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY") 
    QDRANT_URL = "http://localhost:6333"  
    COLLECTION_NAME = "parkinsons_faq"
    EMBEDDING_MODEL = "models/gemini-embedding-001"
    LLM_MODEL = "gemini-2.5-flash"

In [15]:
# def clean_markdown_file(input_path, output_path):
#     with open(input_path, "r", encoding="utf-8") as f:
#         lines = f.readlines()

#     start_idx = None
#     for i, line in enumerate(lines):
#         if "**chapter one**" in line.lower():
#             start_idx = i
#             break
#     if start_idx is None:
#         for i, line in enumerate(lines):
#             if "Appendix: Medications List" in line:
#                 start_idx = i + 1
#                 break
#     if start_idx is None:
#         raise ValueError("Could not find start marker in file.")

#     end_idx = None
#     for i in range(start_idx, len(lines)):
#         if lines[i].strip().lower().startswith("#") and "appendix" in lines[i].lower():
#             end_idx = i
#             break
#     cleaned_lines = lines[start_idx:] if end_idx is None else lines[start_idx:end_idx]

#     result = []
#     i = 0
#     after_question_header = False
#     after_note_header = False
#     in_answer_section = False

#     header_to_remove_pattern = re.compile(r"^#{1,6}\s*\d+\s+frequently asked questions", re.IGNORECASE)

#     while i < len(cleaned_lines):
#         line = cleaned_lines[i]
#         if header_to_remove_pattern.match(line.strip()):
#             prev_idx = len(result) - 1
#             while prev_idx >= 0 and result[prev_idx].strip() == "":
#                 prev_idx -= 1
#             next_idx = i + 1
#             while next_idx < len(cleaned_lines) and cleaned_lines[next_idx].strip() == "":
#                 next_idx += 1
#             if prev_idx >= 0 and next_idx < len(cleaned_lines):
#                 result[prev_idx] = result[prev_idx].rstrip("\n") + " " + cleaned_lines[next_idx].lstrip()
#                 i = next_idx + 1
#                 continue
#             i += 1
#             continue

#         header_match = re.match(r"^(#+)\s+(.+)", line)
#         if header_match:
#             header_level = len(header_match.group(1))
#             header_text = header_match.group(2).strip()

#             if "chapter" in header_text.lower():
#                 clean_text = header_text.replace("*", "").strip()
#                 words = clean_text.split()
#                 if len(words) <= 2 and words[0].lower() == "chapter":
#                     chapter_parts = [header_text]
#                     j = i + 1
#                     while j < len(cleaned_lines):
#                         next_line = cleaned_lines[j].strip()
#                         if not next_line:
#                             j += 1
#                             continue
#                         next_header_match = re.match(r"^(#+)\s+(.+)", next_line)
#                         if next_header_match:
#                             next_level = len(next_header_match.group(1))
#                             next_text = next_header_match.group(2).strip()
#                             if next_level == 1:
#                                 if ("chapter" in next_text.lower() or 
#                                     "?" in next_text or 
#                                     next_text.lower() == "note"):
#                                     break
#                                 chapter_parts.append(next_text)
#                                 j += 1
#                             else:
#                                 break
#                         else:
#                             break
#                     merged_title = " ".join(chapter_parts)
#                     result.append("# " + merged_title + "\n")
#                     i = j - 1
#                 else:
#                     result.append("# " + header_text + "\n")
#                 after_question_header = False
#                 in_answer_section = False
#                 after_note_header = False
#                 i += 1
#                 continue
            
#             if header_level >= 2:
#                 j = i + 1
#                 while j < len(cleaned_lines) and cleaned_lines[j].strip() == "":
#                     j += 1
#                 if j < len(cleaned_lines):
#                     next_line = cleaned_lines[j]
#                     next_header_match = re.match(r"^(#+)\s+(.+)", next_line)
#                     if next_header_match:
#                         next_header_text = next_header_match.group(2).strip()
#                         if "?" in next_header_text:
#                             result.append("## " + header_text + "\n")
#                             i += 1
#                             continue

#             if "?" in header_text:
#                 result.append("### " + header_text + "\n")
#                 after_question_header = True
#                 in_answer_section = True
#                 after_note_header = False
#                 i += 1
#                 continue

#             if header_text.lower() == "note":
#                 result.append("##### NOTE\n")
#                 after_note_header = True
#                 i += 1
#                 continue

#             if after_note_header:
#                 result.append(f"{header_text}\n")
#                 after_note_header = False
#                 i += 1
#                 continue

#             after_note_header = False

#             if in_answer_section:
#                 if "?" in header_text:
#                     pass
#                 else:
#                     result.append("#### " + header_text + "\n")
#                     i += 1
#                     continue

#             if after_question_header:
#                 result.append("#### " + header_text + "\n")
#                 after_question_header = False
#                 i += 1
#                 continue

#             if header_level >= 3:
#                 j = i + 1
#                 while j < len(cleaned_lines) and cleaned_lines[j].strip() == "":
#                     j += 1
#                 if j < len(cleaned_lines):
#                     next_line = cleaned_lines[j]
#                     next_header = re.match(r"^(#+)\s+(.+)", next_line)
#                     if next_header:
#                         result.append("## " + header_text + "\n")
#                         in_answer_section = False
#                         i += 1
#                         continue

#             result.append(line)
#             if header_level <= 2:
#                 in_answer_section = False

#         else:
#             if line.strip():
#                 if after_note_header:
#                     after_note_header = False
#                 if after_question_header and not line.strip() == "":
#                     pass
#             result.append(line)

#         i += 1

#     with open(output_path, "w", encoding="utf-8") as f:
#         f.writelines(result)

In [16]:
# clean_markdown_file("data/interim/frequently-asked-questions-2025/frequently-asked-questions-2025.md", "data/processed/cleaned-parkinsons-faq.md")

In [17]:
loader = TextLoader("data/processed/cleaned-parkinsons-faq.md")
raw_doc = loader.load()[0]

In [18]:
def chunk_document(cleaned_content):
    headers_to_split_on = [
        ("#", "Chapter"),
        ("##", "Section"),
        ("###", "Question")
    ]
    
    # Split only at chapter/section/question levels
    text_splitter = MarkdownHeaderTextSplitter(
        headers_to_split_on=headers_to_split_on,
        strip_headers=True,
        return_each_line=False,
    )
    
    documents = text_splitter.split_text(cleaned_content)
    
    # Process each chunk to extract Q&A format
    processed_docs = []
    for doc in documents:
        metadata = doc.metadata
        content = doc.page_content
        
        # For question chunks
        if "Question" in metadata:
            # Extract answer content including subsections and notes
            answer = re.sub(r'^#{4,5} ?', '', content, flags=re.MULTILINE)
            answer = re.sub(r'\n{3,}', '\n\n', answer).strip()
            
        processed_docs.append(Document(
            page_content=answer,
            metadata={
                "Chapter": metadata.get("Chapter", ""),
                "Section": metadata.get("Section", ""),
                "Question": metadata["Question"],
                "Answer": answer  # 🆕 add answer into metadata
            }
        ))
    
    return processed_docs

In [19]:
with open("data/processed/cleaned-parkinsons-faq.md", "r") as f:
    cleaned_content = f.read()

documents = chunk_document(cleaned_content)

embeddings = GoogleGenerativeAIEmbeddings(
    api_key=Config.GOOGLE_API_KEY,
    model=Config.EMBEDDING_MODEL
)

vector_store = QdrantVectorStore.from_documents(
    documents,
    embeddings,
    url=Config.QDRANT_URL,
    collection_name=Config.COLLECTION_NAME,
    force_recreate=True
)

In [20]:
documents

[Document(metadata={'Chapter': "**chapter one** Understanding Parkinson's", 'Section': 'Overview', 'Question': "What is Parkinson's disease?", 'Answer': "Parkinson's disease (PD) is a progressive neurodegenerative disorder that causes a gradual loss of brain cells that produce dopamine â€” a chemical necessary for movement. As dopamine decreases, movement becomes more difficult and slower.  \nParkinson's is known as a movement disorder because of its more obvious symptoms: tremor, stiffness, slow movements and balance issues. However, non-movement symptoms, including trouble sleeping, depression and speech problems, are extremely common and can be just as challenging."}, page_content="Parkinson's disease (PD) is a progressive neurodegenerative disorder that causes a gradual loss of brain cells that produce dopamine â€” a chemical necessary for movement. As dopamine decreases, movement becomes more difficult and slower.  \nParkinson's is known as a movement disorder because of its more 

In [21]:
def search_faq(vector_store, query: str, k: int = 3) -> List[dict]:
    """
    Perform semantic search and return results with similarity score.
    """
    results = vector_store.similarity_search_with_score(query, k=10)

    # Boost exact match
    boosted = []
    for doc, score in results:
        question = doc.metadata.get("Question", "").lower()
        if query.strip().lower() == question:
            boosted.insert(0, (doc, score))  # put exact match on top
        else:
            boosted.append((doc, score))

    boosted = boosted[:k]

    return [
        {
            "question": doc.metadata.get("Question", ""),
            "answer": doc.page_content.strip(),
            "chapter": doc.metadata.get("Chapter", ""),
            "section": doc.metadata.get("Section", ""),
            "score": score,
        }
        for doc, score in boosted
    ]


In [22]:
test_queries = [
    "So what is Parkinson's disease?",
    "Best way to diagnose Parkinson's disease?",
    "Is there any medications for Parkinson's symptoms",
    "What are the main symptoms of Parkinson's?",
    "Can age affect Parkinson's symptoms?",
    "Can Parkinson's disease ever be cured?",
]

In [23]:
for query in test_queries:
    print(f"\nQuery: {query}")
    results = search_faq(vector_store, query, k=5)
    for idx, item in enumerate(results, 1):
        print(f"\nResult #{idx}")
        print(f"Q: {item['question']}")
        print(f"A: {item['answer']}")
        print(f"Chapter: {item['chapter']} | Section: {item['section']}")
        print(f"Similarity Score: {item['score']:.4f}")


Query: So what is Parkinson's disease?

Result #1
Q: What is Parkinson's disease?
A: Parkinson's disease (PD) is a progressive neurodegenerative disorder that causes a gradual loss of brain cells that produce dopamine â€” a chemical necessary for movement. As dopamine decreases, movement becomes more difficult and slower.  
Parkinson's is known as a movement disorder because of its more obvious symptoms: tremor, stiffness, slow movements and balance issues. However, non-movement symptoms, including trouble sleeping, depression and speech problems, are extremely common and can be just as challenging.
Chapter: **chapter one** Understanding Parkinson's | Section: Overview
Similarity Score: 0.8998

Result #2
Q: Can Parkinson's be cured?
A: No â€” not yet. However, many Parkinson's symptoms can be treated and researchers are making breakthroughs in understanding the disease, its causes and how to further improve therapies.
Chapter: **chapter one** Understanding Parkinson's | Section: Overv

In [24]:
# Print document where question is "What medications are currently available to treat Parkinson's?"
for doc in documents:
    if doc.metadata.get("Question") == "What medications are currently available to treat Parkinson's?":
        print(f"\nDocument for question: {doc.metadata['Question']}")
        print(f"Chapter: {doc.metadata['Chapter']}")
        print(f"Section: {doc.metadata['Section']}")
        print(f"Answer: {doc.page_content.strip()}")
        break


Document for question: What medications are currently available to treat Parkinson's?
Chapter: **chapter two** Treating Parkinson's
Section: Medications & Surgical Treatments
Answer: There are many effective prescription medications for Parkinson's symptoms. It is common for people to take a variety of medications â€” at different doses and times of the day â€” to manage symptoms.  
In the chapter we will discuss:  
- Levodopa
- Dopamine agonists
- COMT (catechol-O-methyl transferase) inhibitors
- MAO-B (monoamine oxidase-B) inhibitors
- Adenosine A2A Antagonists
- Amantadine
- Anticholinergics  
Most Parkinson's medications work by replacing, mimicking or boosting dopamine, the brain chemical that plays an important role in mood, movement and motivation. Dopamine drugs are associated with potential side effects, including nausea, drowsiness, low blood pressure, hallucinations, dyskinesia and compulsive behaviors.  
The choice of medication and dose is often influenced by symptoms, si

In [ ]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.prompts import PromptTemplate

In [40]:
llm = ChatGoogleGenerativeAI(
    model=Config.LLM_MODEL,
    google_api_key=Config.GOOGLE_API_KEY,
    temperature=0.1
)

# Create the prompt template
rag_prompt_template = PromptTemplate(
    input_variables=["question", "context"],
    template="""You are a helpful medical assistant specializing in Parkinson's disease. Answer the user's question based on the provided context from medical FAQs.

Instructions:
- The provided information from the context is from our retrieval results, use the information provided to help answer the question
- No need to mention the source of the information
- If the context doesn't contain enough information to answer the question, say so clearly
- Be accurate and informative
- Use a compassionate and professional tone
- If relevant, mention consulting with healthcare professionals

Context from Parkinson's Disease FAQ:
{context}

Question: {question}

Answer:"""
)

def format_context(search_results: List[dict]) -> str:
    """Format search results into context string for the prompt."""
    context = ""
    for i, result in enumerate(search_results, 1):
        context += f"\n--- FAQ Entry {i} ---\n"
        context += f"Question: {result['question']}\n"
        context += f"Answer: {result['answer']}\n"
        context += f"Source: {result['chapter']} > {result['section']}\n"
    return context.strip()

In [41]:
def rag(query: str, k: int = 3) -> dict:
    """
    Complete RAG pipeline: search, format context, and generate answer using LLM.
    
    Args:
        query: User's question
        k: Number of search results to use as context
        
    Returns:
        Dictionary containing the final answer and metadata
    """
    # Step 1: Retrieve relevant documents
    search_results = search_faq(vector_store, query, k=k)
    
    # Step 2: Format context
    context = format_context(search_results)
    
    # Step 3: Generate prompt
    prompt = rag_prompt_template.format(question=query, context=context)
    
    # Step 4: Get LLM response
    response = llm.invoke(prompt)
    
    # Step 5: Return structured result
    return {
        "question": query,
        "answer": response.content,
        "sources": search_results,
        "context_used": context
    }

In [42]:
test_queries = [
    "So what is Parkinson's disease?",
    "Is there any medications for Parkinson's symptoms",
    "Can Parkinson's disease ever be cured?",
]

In [44]:
print("=" * 80)
print("TESTING RAG PIPELINE WITH LLM")
print("=" * 80)

for query in test_queries:
    print(f"\n{'='*60}")
    print(f"Query: {query}")
    print(f"{'='*60}")
    
    result = rag(query, k=3)
    
    print(f"\nFinal Answer:")
    print(f"{result['answer']}")
    
    print(f"\nSources Used:")
    for i, source in enumerate(result['sources'], 1):
        print(f"{i}. {source['question']} (Score: {source['score']:.4f})")
    
    print("\n" + "-" * 60)

TESTING RAG PIPELINE WITH LLM

Query: So what is Parkinson's disease?

Final Answer:
Parkinson's disease (PD) is a progressive neurodegenerative disorder. It involves a gradual loss of brain cells that produce dopamine, a chemical essential for movement. As dopamine levels decrease, movement becomes more difficult and slower.

While it's often recognized as a movement disorder due to symptoms like tremor, stiffness, slow movements, and balance issues, it's important to know that non-movement symptoms are also very common and can be quite challenging. These can include trouble sleeping, depression, and speech problems.

Sources Used:
1. What is Parkinson's disease? (Score: 0.8998)
2. Can Parkinson's be cured? (Score: 0.8812)
3. How many people are currently living with Parkinson's? (Score: 0.8802)

------------------------------------------------------------

Query: Is there any medications for Parkinson's symptoms

Final Answer:
Yes, there are many effective prescription medications av